<a href="https://colab.research.google.com/github/dru-44/PDA/blob/main/Covid_19_dataset_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Loading the Open COVID-19 Dataset**

---



This very short notebook showcases how to load the Open COVID-19 datset, including some examples for commonly performed operations.

First, loading the data is very simple with pandas. We can use the CSV master table to download the entire Open COVID-19 dataset in a single step:


First, loading the data is very simple with ***pandas***. We can use the CSV master table to download the entire Open COVID-19 dataset in a single step:

In [1]:
import pandas as pd

In [2]:
# Load CSV data directly from the URL with pandas, the options are needed to prevent
# reading of records with key "NA" (Namibia) as NaN
data = pd.read_csv(
    "https://storage.googleapis.com/covid19-open-data/v2/latest/main.csv",
    keep_default_na=False,
    na_values=[""],
)


In [3]:
# NOTE: We are only downloading the last day of data, but you can download all

# Print a small snippet of the dataset
print(f"The dataset currently contains {len(data)} records, here is a sample:")
data.sample(5)  #sample(5) outputs 5 records.

The dataset currently contains 21411 records, here is a sample:


,key,date,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,locality_code,locality_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator,population,population_male,population_female,rural_population,urban_population,largest_city_population,clustered_population,population_density,...,pollution_mortality_rate,comorbidity_mortality_rate,hospital_beds,nurses,physicians,health_expenditure,out_of_pocket_health_expenditure,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,stringency_index,noaa_station,noaa_distance,average_temperature,minimum_temperature,maximum_temperature,rainfall,snowfall,dew_point,relative_humidity
10369,IL_D_1773,2020-11-27,NaN,NaN,IL,Israel,D,Southern District,1773,מפעלי מישור רותם,NaN,NaN,IL,ISR,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2436,BR_MG_313130,2020-11-27,Q318314,NaN,BR,Brazil,MG,Minas Gerais,313130,Ipatinga,NaN,NaN,BR,BRA,2,56.0,5.0,NaN,NaN,11156.0,249.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,265409.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-11.0,23.0,14.0,51.0,0.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.680110e+10,16.298938,22.708333,19.423611,26.541667,0.000000,NaN,17.465278,72.660513
4089,BR_PR_411050,2020-11-27,Q2064845,NaN,BR,Brazil,PR,Paraná,411050,Ipiranga,NaN,NaN,BR,BRA,2,0.0,0.0,NaN,NaN,141.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14150.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.693010e+10,26.832715,18.930556,13.611111,26.104167,0.870857,NaN,13.201389,70.098436
1642,BR_GO_520310,2020-11-27,Q805016,NaN,BR,Brazil,GO,Goiás,520310,Baliza,NaN,NaN,BR,BRA,2,1.0,0.0,NaN,NaN,40.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.335910e+10,40.806905,29.103175,22.785714,35.492063,0.000000,NaN,13.293651,38.090165
3113,BR_MT_510685,2020-11-27,Q738627,NaN,BR,Brazil,MT,Mato Grosso,510685,Porto Estrela,NaN,NaN,BR,BRA,2,0.0,0.0,NaN,NaN,131.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3064.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.670410e+10,0.227268,29.952381,23.539683,37.428571,0.000000,NaN,13.404762,45.410717


Looking at country-level data
Some records contain country-level data, in other words, data that is aggregated at the country level. Other records contain region-level data, which are subdivisions of a country; for example, Chinese provinces or USA states. A few regions also report at an even smaller subdivision, i.e. county/municipality level.

To filter only country-level data from the dataset, look for records that have a *aggregation_level == 0* or, alternatively, null value for the *subregion1_code* (or *subregion1_name*) field:

In [4]:
# Look for rows with country level data

countries = data[data.aggregation_level == 0]

# We no longer need the subregion-level columns
countries = countries.drop(columns=['subregion1_code', 'subregion1_name', 'subregion2_code', 'subregion2_name'])

countries.tail()

,key,date,wikidata,datacommons,country_code,country_name,locality_code,locality_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator,population,population_male,population_female,rural_population,urban_population,largest_city_population,clustered_population,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,...,pollution_mortality_rate,comorbidity_mortality_rate,hospital_beds,nurses,physicians,health_expenditure,out_of_pocket_health_expenditure,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,stringency_index,noaa_station,noaa_distance,average_temperature,minimum_temperature,maximum_temperature,rainfall,snowfall,dew_point,relative_humidity
21344,YE,2020-11-27,Q805,country/YEM,YE,Yemen,NaN,NaN,YE,YEM,0,10.0,2.0,NaN,NaN,2124.0,611.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,29161922.0,14692284.0,14469638.0,18292399.0,10869523.0,2874373.0,2874373.0,53.977853,0.452,8172997.0,6846631.0,5799728.0,...,194.2,30.6,NaN,0.7852,NaN,72.039467,58.321163,23.0,39.0,28.0,10.0,1.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,24.07,4.113610e+10,238.027608,22.944444,13.222222,29.722222,0.000000,NaN,3.722222,28.521439
21346,YT,2020-11-27,Q17063,country/MYT,YT,Mayotte,NaN,NaN,YT,MYT,0,0.0,0.0,NaN,NaN,5122.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,256518.0,109092.0,115551.0,NaN,NaN,NaN,NaN,NaN,NaN,59419.0,53241.0,36178.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.700510e+10,16.041561,28.703704,24.240741,32.037037,0.000000,NaN,23.666667,74.265833
21347,ZA,2020-11-27,Q258,country/ZAF,ZA,South Africa,NaN,NaN,ZA,ZAF,0,3250.0,118.0,NaN,29437.0,775502.0,21201.0,NaN,5355068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58558270.0,28859303.0,29698967.0,19408553.0,39149717.0,5635127.0,20836302.0,47.630120,0.699,11887295.0,11028683.0,10950688.0,...,86.7,26.2,NaN,1.3078,0.9054,499.237549,38.775391,-14.0,1.0,-30.0,-25.0,-19.0,11.0,0.0,1.0,1.0,2.0,0.0,2.0,0.0,2.0,1.0,2.0,0.0,0.0,2.0,3.0,2.0,0.0,0.0,38.89,6.843810e+10,77.648653,21.870370,15.030864,30.154321,26.867556,NaN,14.129630,61.980737
21409,ZM,2020-11-27,Q953,country/ZMB,ZM,Zambia,NaN,NaN,ZM,ZMB,0,69.0,0.0,NaN,4168.0,17535.0,357.0,NaN,378191.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17861030.0,8843212.0,9017818.0,9989317.0,7871713.0,2646619.0,2646619.0,23.341479,0.588,5799199.0,4507836.0,3303315.0,...,127.2,17.9,NaN,1.3376,1.1867,67.648666,8.015779,1.0,7.0,0.0,-11.0,-1.0,3.0,1.0,2.0,1.0,4.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,3.0,2.0,0.0,0.0,45.37,6.766510e+10,155.975684,25.666667,21.000000,31.222222,0.000000,NaN,15.722222,54.191622
21410,ZW,2020-11-27,Q954,country/ZWE,ZW,Zimbabwe,NaN,NaN,ZW,ZWE,0,110.0,0.0,NaN,923.0,9508.0,274.0,NaN,161776.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14645468.0,6983351.0,7662117.0,9928163.0,4717305.0,1521311.0,1521311.0,37.324591,0.535,4056369.0,3235339.0,2602286.0,...,133.0,19.3,NaN,1.9346,0.2096,110.149620,22.715431,8.0,23.0,17.0,5.0,9.0,4.0,1.0,1.0,2.0,3.0,1.0,2.0,2.0,2.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,67.59,6.777510e+10,165.789567,22.000000,17.000000,25.000000,0.000000,NaN,8.833333,42.961386


Looking at state/province data
Conversely, to filter state/province data for a specific country, we need to look for records where the aggregation level is **1** (or where the region columns have non-null values). The following snippet extracts data related to Spain's subregions from the dataset:

In [5]:
# Filter records that have the right country code AND a non-null region code
# Same as `data[(data.country_code == 'ES') & ~(data.subregion`_code.isna())]`
spain_regions = data[(data.country_code == 'ES') & (data.aggregation_level == 1)]

# We no longer need the municipality-level columns
spain_regions = spain_regions.drop(columns=['subregion2_code', 'subregion2_name'])

spain_regions.tail()

,key,date,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,locality_code,locality_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator,population,population_male,population_female,rural_population,urban_population,largest_city_population,clustered_population,population_density,human_development_index,population_age_00_09,...,pollution_mortality_rate,comorbidity_mortality_rate,hospital_beds,nurses,physicians,health_expenditure,out_of_pocket_health_expenditure,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,stringency_index,noaa_station,noaa_distance,average_temperature,minimum_temperature,maximum_temperature,rainfall,snowfall,dew_point,relative_humidity
9657,ES_ML,2020-11-27,Q5831,nuts/ES64,ES,Spain,ML,Melilla,NaN,NaN,ES,ESP,1,0.0,0.0,NaN,NaN,3340.0,34.0,NaN,NaN,0.0,44.0,NaN,0.0,3.0,NaN,NaN,NaN,NaN,84689.0,42793.0,41896.0,NaN,NaN,NaN,NaN,6049.9,NaN,14027.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-44.0,-8.0,-35.0,-37.0,-14.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.033810e+10,0.820365,13.383333,7.144444,18.661111,0.000000,NaN,5.622222,59.505137
9658,ES_NC,2020-11-27,Q4018,nuts/ES22,ES,Spain,NC,Navarra,NaN,NaN,ES,ESP,1,0.0,3.0,NaN,NaN,36380.0,810.0,NaN,NaN,0.0,2048.0,NaN,0.0,136.0,NaN,NaN,NaN,NaN,649946.0,321064.0,328882.0,NaN,NaN,NaN,NaN,62.5,0.918,65240.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-45.0,19.0,-9.0,-18.0,-15.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.085100e+09,5.201020,6.166667,-0.135802,14.364198,0.056444,10.16,1.882716,74.253882
9659,ES_PV,2020-11-27,Q3995,nuts/ES21,ES,Spain,PV,País Vasco,NaN,NaN,ES,ESP,1,0.0,0.0,NaN,NaN,87053.0,2331.0,NaN,NaN,8.0,7032.0,NaN,0.0,578.0,NaN,NaN,NaN,NaN,2177880.0,1054101.0,1123779.0,NaN,NaN,NaN,NaN,302.4,0.924,197465.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-51.0,17.0,7.0,-14.0,-21.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.080100e+09,14.175633,6.488889,0.333333,14.516667,0.050800,10.16,1.700000,72.089940
9660,ES_RI,2020-11-27,Q5727,nuts/ES23,ES,Spain,RI,La Rioja,NaN,NaN,ES,ESP,1,0.0,3.0,NaN,NaN,15778.0,521.0,NaN,NaN,5.0,1504.0,NaN,0.0,91.0,NaN,NaN,NaN,NaN,313571.0,154512.0,159059.0,NaN,NaN,NaN,NaN,62.3,NaN,29152.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-39.0,4.0,-14.0,-35.0,-14.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.084100e+09,26.140786,5.561111,-1.066667,14.111111,0.000000,10.16,-0.038889,67.774783
9661,ES_VC,2020-11-27,Q5720,nuts/ES52,ES,Spain,VC,Comunidad Valenciana,NaN,NaN,ES,ESP,1,0.0,4.0,NaN,NaN,93244.0,2108.0,NaN,NaN,5.0,5747.0,NaN,3.0,730.0,NaN,NaN,NaN,NaN,4974969.0,2450794.0,2524175.0,NaN,NaN,NaN,NaN,214.4,NaN,467218.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-24.0,-4.0,-16.0,-27.0,-14.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.284100e+09,23.036144,9.512346,4.037037,16.987654,0.000000,10.16,4.641975,72.098709


Using the key column
The key column is present in all datasets and is unique for each combination of country, province/state and municipality/county. This way, we can retrieve a specific country or region using a single filter for the data. The key column is built using country_code for country-level data, ${country_code}_${subregion1_code} for province/state level data, and ${country_code}_${subregion1_code}_${subregion2_code} for municipality/county data:

In [8]:
# Filter records for Spain at the country-level
spain_country = data[data.key == 'ES']

# We no longer need the subregion-level columns
spain_country = spain_country.drop(columns=['subregion1_code', 'subregion1_name', 'subregion2_code', 'subregion2_name'])

spain_country.tail()  #tail outputs last records

,key,date,wikidata,datacommons,country_code,country_name,locality_code,locality_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator,population,population_male,population_female,rural_population,urban_population,largest_city_population,clustered_population,population_density,human_development_index,population_age_00_09,population_age_10_19,population_age_20_29,...,pollution_mortality_rate,comorbidity_mortality_rate,hospital_beds,nurses,physicians,health_expenditure,out_of_pocket_health_expenditure,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,stringency_index,noaa_station,noaa_distance,average_temperature,minimum_temperature,maximum_temperature,rainfall,snowfall,dew_point,relative_humidity
8264,ES,2020-11-27,Q29,country/ESP,ES,Spain,NaN,NaN,ES,ESP,0,4386.0,75.0,NaN,NaN,1594844.0,43668.0,NaN,NaN,140.0,124521.0,NaN,20.0,11454.0,NaN,NaN,NaN,NaN,46937060.0,23009259.0,23927801.0,9149372.0,37927409.0,6559041.0,12100168.0,93.1,0.876,4424389.0,4837889.0,4813442.0,...,9.9,9.9,NaN,5.7295,3.8723,2506.464844,590.668274,-40.0,0.0,-13.0,-27.0,-20.0,10.0,1.0,2.0,2.0,4.0,0.0,2.0,2.0,3.0,2.0,2.0,0.0,0.0,2.0,2.0,2.0,0.0,0.0,71.3,8.224100e+09,21.70187,7.722222,2.016667,14.433333,0.0,10.16,1.338889,65.161173


In [9]:
# Filter records for Madrid, one of the subregions of Spain
madrid = data[data.key == 'ES_MD']

madrid.tail()

,key,date,wikidata,datacommons,country_code,country_name,subregion1_code,subregion1_name,subregion2_code,subregion2_name,locality_code,locality_name,3166-1-alpha-2,3166-1-alpha-3,aggregation_level,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested,new_hospitalized,total_hospitalized,current_hospitalized,new_intensive_care,total_intensive_care,current_intensive_care,new_ventilator,total_ventilator,current_ventilator,population,population_male,population_female,rural_population,urban_population,largest_city_population,clustered_population,population_density,...,pollution_mortality_rate,comorbidity_mortality_rate,hospital_beds,nurses,physicians,health_expenditure,out_of_pocket_health_expenditure,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,mobility_transit_stations,mobility_workplaces,mobility_residential,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,income_support,debt_relief,fiscal_measures,international_support,public_information_campaigns,testing_policy,contact_tracing,emergency_investment_in_healthcare,investment_in_vaccines,stringency_index,noaa_station,noaa_distance,average_temperature,minimum_temperature,maximum_temperature,rainfall,snowfall,dew_point,relative_humidity
9456,ES_MD,2020-11-27,Q5756,nuts/ES30,ES,Spain,MD,Comunidad de Madrid,NaN,NaN,NaN,NaN,ES,ESP,1,0.0,1.0,NaN,NaN,299636.0,11040.0,NaN,NaN,75.0,42497.0,NaN,7.0,3617.0,NaN,NaN,NaN,NaN,6641649.0,3184373.0,3457276.0,NaN,NaN,NaN,NaN,827.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-32.0,-4.0,-13.0,-32.0,-28.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.222100e+09,1.123194,7.722222,2.016667,14.433333,0.0,10.16,1.338889,65.161173


Dataset Subsets

---


The master table can be large and cumbersome depending on your application. If you only need a subset of the data, you can consult each table individually. For example, here's how you would get only epidemiology data for Madrid:

In [10]:
# Load the epidemiology table(Another table)
# Note that all the helper columns such as country code, country name, aggregation level, etc. are present in the
# `index` table; we only have the key here
epi = pd.read_csv('https://storage.googleapis.com/covid19-open-data/v2/epidemiology.csv')

# Filter records for Madrid, one of the subregions of Spain
madrid = epi[epi.key == 'ES_MD']

madrid.tail()

,date,key,new_confirmed,new_deceased,new_recovered,new_tested,total_confirmed,total_deceased,total_recovered,total_tested
3566162,2020-11-21,ES_MD,167.0,NaN,NaN,NaN,298948.0,NaN,NaN,NaN
3579429,2020-11-22,ES_MD,100.0,NaN,NaN,NaN,299048.0,NaN,NaN,NaN
3593225,2020-11-23,ES_MD,145.0,NaN,NaN,NaN,299193.0,NaN,NaN,NaN
3606355,2020-11-24,ES_MD,443.0,NaN,NaN,NaN,299636.0,NaN,NaN,NaN
3619022,2020-11-25,ES_MD,0.0,NaN,NaN,NaN,299636.0,NaN,NaN,NaN


Data consistency

---


Often, region-level data and country-level data will come from different 
sources. This will lead to numbers not adding up exactly, or even date misalignment (the data for the region may be reported sooner or later than the whole country). However, country- and region- level data will always be self-consistent